# Librairies

In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data

https://www.data.gouv.fr/fr/datasets/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012/

The excel file has two types of sheets : national police & local one.

## Understanding one type of sheet : National Police in 2012

In [210]:
#For the national police on 2012
NP_2012=pd.read_excel("crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx",sheet_name=2).rename({"Départements":"crimeType"},axis=1).set_index("crimeType")

In [211]:
NP_2012.head()

,01,01.1,01.2,02,02.1,02.2,02.3,02.4,03,03.1,...,974.1,974.2,974.3,974.4,976,976.1,987,987.1,988,988.1
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Homicides pour d'autres motifs,0,0,1,0,2,0,0,0,0,0,...,7,0,1,2,0,1,0,0,0,7
Tentatives d'homicides pour voler et à l'occasion de vols,0,0,0,0,0,1,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Tentatives homicides pour d'autres motifs,0,0,0,0,4,1,0,0,0,1,...,3,0,1,0,0,1,0,0,0,1


### Overall sum

We will sum for each year all crimes in France.

In [212]:
NP_2012.sum(axis=1)

crimeType
Règlements de compte entre malfaireurs                          34
Homicides pour voler et à l'occasion de vols                    14
Homicides pour d'autres motifs                                 387
Tentatives d'homicides pour voler et à l'occasion de vols       68
Tentatives homicides pour d'autres motifs                      912
                                                             ...  
Infractions à l'exercice d'une profession règlementée          601
Infractions au droit de l'urbanisme et de la construction      833
Fraudes fiscales                                               524
Autres délits économiques et financiers                       9530
Autres délits                                                65181
Length: 107, dtype: int64

### Group by departement

The case of the Corse makes it tricky. All departements are labelled with numbers from 1 to 992, but for this island, we don't have numbers but '2A' and '2B'. 

In [213]:
columns=NP_2012.columns
lenDF=len(columns)

dfSum=pd.DataFrame(index=NP_2012.index)
NP_2012IndexC=NP_2012.copy()

countDep=1 #counter for the departements 
countLines=0

depDf=pd.DataFrame(index=NP_2012.index)

A=False
B=False

We have two specific cases to deal with:
* gap from dep n° 19 to 21,
* gap from 29 to 2A&2B to 31. 

In [214]:
for index, value in enumerate(columns):

    if value=='30': 
        countDep=30

    if value in ['2A','2A.1','2A.2','2A.3','2A.4']: #first case of Corse
        
            newCol=1000+index
            NP_2012[str(newCol)]= NP_2012.iloc[:,index] #adding at the end a column
            
            NP_2012=NP_2012.drop(str(value),axis=1)
            depDf=pd.DataFrame(index=NP_2012.index)

            if columns[index+1]=='2B':
                A=True
        

    elif value in ['2B','2B.1','2B.2','2B.3','2B.4']: #second case of Corse
       
            newCol=2000+index
            NP_2012[str(newCol)]= NP_2012.iloc[:,index] #adding at the end a column
            NP_2012=NP_2012.drop(str(value),axis=1)
            depDf=pd.DataFrame(index=NP_2012.index)

            if columns[index+1]=='30':
                B=True

    
    elif ( countDep<=float(value) )  and ( float(value) < countDep+1 ) :
        
        depDf[str(index)]=NP_2012.iloc[:,index]

        try:
            if float(columns[index+1])==21 and countDep+1==20:
                countDep+=1
        except:
            continue

        if columns[index+1]=='2A':

            countDep=29
            dfSum[str(countDep)]=depDf.sum(axis=1)  
            depDf=pd.DataFrame(index=NP_2012.index)

        if columns[index+1]!='2A':
            if float(columns[index+1])==countDep+1 or float(columns[index+1])==971 and float(value)!=1 :
                
                dfSum[str(countDep)]=depDf.sum(axis=1) 
                countDep+=1
                depDf=pd.DataFrame(index=NP_2012.index)

try:  76 19.2 20
error


In [215]:
dfSum

,1,2,3,4,5,6,7,8,9,10,...,86,87,88,89,90,91,92,93,94,95
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,3,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Homicides pour d'autres motifs,1,2,2,0,0,22,0,0,0,0,...,1,0,1,0,0,4,15,16,4,16
Tentatives d'homicides pour voler et à l'occasion de vols,0,1,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,8,4,1,0
Tentatives homicides pour d'autres motifs,0,5,2,0,0,12,0,0,0,0,...,2,0,1,0,3,39,44,64,36,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,4,4,0,2,2,17,0,2,2,2,...,0,0,1,2,0,10,17,8,55,2
Infractions au droit de l'urbanisme et de la construction,3,2,2,0,1,169,0,0,0,2,...,0,1,5,1,0,9,4,10,19,3
Fraudes fiscales,1,3,0,0,0,20,0,0,0,1,...,0,1,1,2,0,4,23,3,9,4


### Adding the Corse island

In [216]:
dfSumCorse=dfSum.copy()

In [217]:
dfSumCorse.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
       '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
       '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
       '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86',
       '87', '88', '89', '90', '91', '92', '93', '94', '95'],
      dtype='object')

In [218]:
res=[float(ele) for ele in NP_2012.columns]

In [219]:
A=[1000<=ele<2000 for ele in res]
B=[2000<=ele<3000 for ele in res]

I set arbitrarly 991 and 992 for the two departements of the Corse. 

In [220]:
dfSumCorse['991']=NP_2012IndexC.iloc[:,A].sum(axis=1)
dfSumCorse['992']=NP_2012IndexC.iloc[:,B].sum(axis=1)

In [221]:
dfSumCorse

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,991,992
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,3,0,0,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Homicides pour d'autres motifs,1,2,2,0,0,22,0,0,0,0,...,1,0,0,4,15,16,4,16,1,7
Tentatives d'homicides pour voler et à l'occasion de vols,0,1,0,0,0,5,0,0,0,0,...,0,0,0,0,8,4,1,0,0,0
Tentatives homicides pour d'autres motifs,0,5,2,0,0,12,0,0,0,0,...,1,0,3,39,44,64,36,26,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,4,4,0,2,2,17,0,2,2,2,...,1,2,0,10,17,8,55,2,7,2
Infractions au droit de l'urbanisme et de la construction,3,2,2,0,1,169,0,0,0,2,...,5,1,0,9,4,10,19,3,1,0
Fraudes fiscales,1,3,0,0,0,20,0,0,0,1,...,1,2,0,4,23,3,9,4,0,0


## Generalization

We will :
* create a sheet with the sum of crimes for each year,
* create one sheet a year with all departements.

### Open every file

In [222]:
def read_file(i):
    return pd.read_excel("crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx",sheet_name=i).rename({"Départements":"crimeType"},axis=1).set_index("crimeType")

In [223]:
NP_2012=read_file(1)
NP_2013=read_file(2)
NP_2014=read_file(3)
NP_2015=read_file(4)
NP_2016=read_file(5)
NP_2017=read_file(6)
NP_2018=read_file(7)
NP_2019=read_file(8)
NP_2020=read_file(9)

In [224]:
#For the local police
LP_2012=read_file(10)
LP_2013=read_file(11)
LP_2014=read_file(12)
LP_2015=read_file(13)
LP_2016=read_file(14)
LP_2017=read_file(15)
LP_2018=read_file(16)
LP_2019=read_file(17)
LP_2020=read_file(18)

### Dataframe with the number of crime by year

In [225]:
dfByYear=pd.DataFrame()

In [226]:
dfByYear["2012"]=NP_2012.sum(axis=1)
dfByYear["2013"]=NP_2013.sum(axis=1)
dfByYear["2014"]=NP_2014.sum(axis=1)
dfByYear["2015"]=NP_2015.sum(axis=1)
dfByYear["2016"]=NP_2016.sum(axis=1)
dfByYear["2017"]=NP_2017.sum(axis=1)
dfByYear["2018"]=NP_2018.sum(axis=1)
dfByYear["2019"]=NP_2019.sum(axis=1)
dfByYear["2020"]=NP_2020.sum(axis=1)

In [227]:
dfByYear

,2012,2013,2014,2015,2016,2017,2018,2019,2020
crimeType,,,,,,,,,
Règlements de compte entre malfaireurs,50,34,45,42,59,84,93,71,82
Homicides pour voler et à l'occasion de vols,23,14,15,9,22,19,23,19,24
Homicides pour d'autres motifs,377,387,399,412,465,459,424,514,496
Tentatives d'homicides pour voler et à l'occasion de vols,47,68,43,67,62,74,76,73,62
Tentatives homicides pour d'autres motifs,1035,912,1055,1430,1638,1719,1820,2007,2121
...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,624,601,562,1199,1196,970,1320,1292,760
Infractions au droit de l'urbanisme et de la construction,899,833,999,1263,1335,1247,1248,1120,896
Fraudes fiscales,617,524,549,341,384,385,377,412,462


### Group by departements for each year

In [228]:
def dfGroupByDep(df):

    data=df.copy()

    columns=data.columns
    lenDF=len(columns)

    dfOneYearAllDeps=pd.DataFrame(index=data.index)
    dfOneDep=pd.DataFrame(index=data.index)

    countDep=1 #counter for the departements 
    countLines=0

    A=False
    B=False

    for index, value in enumerate(columns):

        if value=='30':
            countDep=30

        if value in ['2A','2A.1','2A.2','2A.3','2A.4']: #first case of Corse

                newCol=1000+index
                data[str(newCol)]= data.iloc[:,index] #adding at the end a column

                data=data.drop(str(value),axis=1)
                dfOneDep=pd.DataFrame(index=data.index)

                if columns[index+1]=='2B':
                    A=True

        elif value in ['2B','2B.1','2B.2','2B.3','2B.4']: #second case of Corse
        
                newCol=2000+index
                data[str(newCol)]= data.iloc[:,index] #adding at the end a column
                data=data.drop(str(value),axis=1)

                dfOneDep=pd.DataFrame(index=data.index)

                if columns[index+1]=='30':
                    B=True


        elif ( countDep<=float(value) )  and ( float(value) < countDep+1 ) :

            dfOneDep[str(index)]=data.iloc[:,index]

            try:
                if float(columns[index+1])==21 and countDep+1==20:
                    countDep+=1
            except:
                continue

            if columns[index+1]=='2A':

                countDep=29
                dfOneYearAllDeps[str(countDep)]=dfOneDep.sum(axis=1)  

                dfOneDep=pd.DataFrame(index=data.index)

            if columns[index+1]!='2A':
                if float(columns[index+1])==countDep+1 or float(columns[index+1])==971 and float(value)!=1 :

                    dfOneYearAllDeps[str(countDep)]=dfOneDep.sum(axis=1)
                    countDep+=1

                    dfOneDep=pd.DataFrame(index=data.index)            

    res=[float(ele) for ele in data.columns]

    A=[1000<=ele<2000 for ele in res]
    B=[2000<=ele<3000 for ele in res]

    dfOneYearAllDeps['991']=data.iloc[:,A].sum(axis=1)
    dfOneYearAllDeps['992']=data.iloc[:,B].sum(axis=1)


    return dfOneYearAllDeps

In [229]:
dfByDep2012=dfGroupByDep(NP_2012)
dfByDep2013=dfGroupByDep(NP_2013)
dfByDep2014=dfGroupByDep(NP_2014)
dfByDep2015=dfGroupByDep(NP_2015)
dfByDep2016=dfGroupByDep(NP_2016)
dfByDep2017=dfGroupByDep(NP_2017)
dfByDep2018=dfGroupByDep(NP_2018)
dfByDep2019=dfGroupByDep(NP_2019)
dfByDep2020=dfGroupByDep(NP_2020)

In [230]:
dfByDep2020

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,991,992
crimeType,,,,,,,,,,,,,,,,,,,,,
Règlements de compte entre malfaireurs,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,4,2,1,4,0,0
Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12,0,0
Homicides pour d'autres motifs,0,6,2,1,0,16,2,2,1,1,...,0,1,1,5,24,24,14,21,1,0
Tentatives d'homicides pour voler et à l'occasion de vols,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,5,11,2,6,0,0
Tentatives homicides pour d'autres motifs,1,6,19,0,2,49,0,2,0,3,...,0,3,2,43,104,146,58,52,3,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Infractions à l'exercice d'une profession règlementée,1,0,3,0,0,7,0,0,0,2,...,0,2,0,10,18,13,36,8,1,0
Infractions au droit de l'urbanisme et de la construction,3,3,2,1,1,73,0,1,1,4,...,0,0,0,21,14,19,21,16,4,3
Fraudes fiscales,0,0,1,0,1,11,0,0,0,0,...,0,0,0,27,19,17,29,13,0,1
